In [1]:
#install mlxtend package by opening cmd and
#pip install mlxtend

import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
df = pd.read_csv('france_data.csv')
df.head()

InvoiceNo StockCode                      Description  Quantity  \
0    536370     22728        ALARM CLOCK BAKELIKE PINK        24   
1    536370     22727        ALARM CLOCK BAKELIKE RED         24   
2    536370     22726       ALARM CLOCK BAKELIKE GREEN        12   
3    536370     21724  PANDA AND BUNNIES STICKER SHEET        12   
4    536370     21883                 STARS GIFT TAPE         24   

      InvoiceDate  UnitPrice  CustomerID Country  
0  12/1/2010 8:45       3.75     12583.0  France  
1  12/1/2010 8:45       3.75     12583.0  France  
2  12/1/2010 8:45       3.75     12583.0  France  
3  12/1/2010 8:45       0.85     12583.0  France  
4  12/1/2010 8:45       0.65     12583.0  France

In [3]:
df.shape

(8557, 8)

In [4]:
#number of unique transactions
len(df.groupby(['InvoiceNo'])['Quantity'].count())

461

In [5]:
#highest billed customer#
max_buy = pd.DataFrame(df.groupby(['CustomerID'])['UnitPrice'].sum().reset_index())
max_buy = max_buy.sort_values(by='UnitPrice', ascending=False)
max_buy.head()

CustomerID  UnitPrice
16     12536.0   13255.22
48     12669.0    2524.18
55     12681.0    2061.71
56     12682.0    1968.83
20     12567.0    1557.35

In [6]:
#taking a look at the purchases made by customer
highest_cust = df[df['CustomerID']==12536]
highest_cust = highest_cust.sort_values(by='UnitPrice', ascending=False)
highest_cust.head()
#it is seen that there are two products whose Description is Manual and there price is high
#lesson: look at the data carefully before believing it

InvoiceNo StockCode                 Description  Quantity  \
6531   C573079         M                      Manual        -2   
6532    573080         M                      Manual         1   
6530    573077         M                      Manual         1   
6213    570672     22846  BREAD BIN DINER STYLE RED          1   
6231    570672     23007      SPACEBOY BABY GIFT SET         3   

           InvoiceDate  UnitPrice  CustomerID Country  
6531  10/27/2011 14:15    4161.06     12536.0  France  
6532  10/27/2011 14:20    4161.06     12536.0  France  
6530  10/27/2011 14:13    4161.06     12536.0  France  
6213  10/11/2011 14:52      16.95     12536.0  France  
6231  10/11/2011 14:52      16.95     12536.0  France

In [7]:
#second highest billed customer
sec_cust = df[df['CustomerID']==12681]
sec_cust = sec_cust.sort_values(by='UnitPrice', ascending = False)
sec_cust.head()

InvoiceNo StockCode Description  Quantity       InvoiceDate  UnitPrice  \
7991   C578743      POST     POSTAGE        -3  11/25/2011 11:35       18.0   
3831    559418      POST     POSTAGE         3    7/8/2011 11:36       18.0   
7632    577121      POST     POSTAGE         4  11/17/2011 18:29       18.0   
1113    542735      POST     POSTAGE         3   1/31/2011 15:36       18.0   
474     540178      POST     POSTAGE         3    1/5/2011 12:42       18.0   

      CustomerID Country  
7991     12681.0  France  
3831     12681.0  France  
7632     12681.0  France  
1113     12681.0  France  
474      12681.0  France

In [8]:
#creating a user*product matrix
basket = pd.pivot_table(df,index='InvoiceNo', columns='Description',values='Quantity', fill_value=0)
basket.shape

(461, 1565)

In [9]:
basket

Description   50'S CHRISTMAS GIFT BAG LARGE   DOLLY GIRL BEAKER  \
InvoiceNo                                                         
536370                                    0                   0   
536852                                    0                   0   
536974                                    0                   0   
537065                                    0                   0   
537463                                    0                   0   
537468                                    0                   0   
537693                                    0                   0   
537897                                    0                   0   
537967                                    0                   0   
538008                                    0                   0   
538093                                    0                   0   
538196                                    0                   0   
539050                                    0                   0   
539113                                    0                   0   
539407                                    0                   0   
539435                                    0                   0   
539551                                    0                   0   
539607                                    0                   0   
539688                                    0                   0   
539727                                    0                   0   
539829                                    0                   0   
540178                                    0                   0   
540239                                    0                   0   
540351                                    0                   0   
540365                                    0                   0   
540455                                    0                   0   
540463                                    0                   0   
540521                                    0                   0   
540642                                    0                   0   
540688                                    0                   0   
...                                     ...                 ...   
C565615                                   0                   0   
C566960                                   0                   0   
C567091                                   0                   0   
C567916                                   0                   0   
C568244                                   0                   0   
C568534                                   0                   0   
C568557                                   0                   0   
C569981                                   0                   0   
C570291                                   0                   0   
C570515                                   0                   0   
C571127                                   0                   0   
C571473                                   0                   0   
C571678                                   0                   0   
C571680                                   0                   0   
C573079                                   0                   0   
C573869                                   0                   0   
C574095                                   0                   0   
C574261                                   0                   0   
C574512                                   0                   0   
C574946                                   0                   0   
C576908                                   0                   0   
C578377                                   0                   0   
C578743                                   0                   0   
C579127                                   0                   0   
C579192                                   0                   0   
C579532                                   0                   0   
C579562                                   0    

In [10]:
#function to replace natural number with 1
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets.drop('POSTAGE', inplace=True, axis=1) #dropping off the records which have been posted


In [11]:
#applying apriori algorithm to generate rules which have support more than 0.05
frequent_itemsets = apriori(basket_sets, min_support=0.05, use_colnames=True)
frequent_itemsets.head()

support                       itemsets
0  0.060738  [4 TRADITIONAL SPINNING TOPS]
1  0.082430   [ALARM CLOCK BAKELIKE GREEN]
2  0.086768    [ALARM CLOCK BAKELIKE PINK]
3  0.080260    [ALARM CLOCK BAKELIKE RED ]
4  0.058568   [ASSORTED COLOUR MINI CASES]

In [12]:
#calculating confidence and lift for the genrated rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

antecedants                   consequents  \
0  (ALARM CLOCK BAKELIKE GREEN)   (ALARM CLOCK BAKELIKE PINK)   
1   (ALARM CLOCK BAKELIKE PINK)  (ALARM CLOCK BAKELIKE GREEN)   
2  (ALARM CLOCK BAKELIKE GREEN)   (ALARM CLOCK BAKELIKE RED )   
3   (ALARM CLOCK BAKELIKE RED )  (ALARM CLOCK BAKELIKE GREEN)   
4   (ALARM CLOCK BAKELIKE PINK)   (ALARM CLOCK BAKELIKE RED )   

   antecedent support  consequent support   support  confidence       lift  \
0            0.082430            0.086768  0.062907    0.763158   8.795395   
1            0.086768            0.082430  0.062907    0.725000   8.795395   
2            0.082430            0.080260  0.067245    0.815789  10.164296   
3            0.080260            0.082430  0.067245    0.837838  10.164296   
4            0.086768            0.080260  0.062907    0.725000   9.033108   

   leverage  conviction  
0  0.055754    3.855869  
1  0.055754    3.336620  
2  0.060629    4.992873  
3  0.060629    5.658351  
4  0.055943    3.344508

In [13]:
#selecting only the rules which have lift >6 and confidence >0.8
rules[ (rules['lift'] >= 6) &
       (rules['confidence'] >= 0.8) ]

antecedants  \
2                        (ALARM CLOCK BAKELIKE GREEN)   
3                         (ALARM CLOCK BAKELIKE RED )   
6                       (CHILDRENS CUTLERY SPACEBOY )   
7                     (CHILDRENS CUTLERY DOLLY GIRL )   
32                      (SET/6 RED SPOTTY PAPER CUPS)   
33                    (SET/6 RED SPOTTY PAPER PLATES)   
34  (ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...   
35  (ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...   
36  (ALARM CLOCK BAKELIKE PINK, ALARM CLOCK BAKELI...   
46  (SET/20 RED RETROSPOT PAPER NAPKINS , SET/6 RE...   
47  (SET/20 RED RETROSPOT PAPER NAPKINS , SET/6 RE...   
48  (SET/6 RED SPOTTY PAPER CUPS, SET/6 RED SPOTTY...   

                              consequents  antecedent support  \
2             (ALARM CLOCK BAKELIKE RED )            0.082430   
3            (ALARM CLOCK BAKELIKE GREEN)            0.080260   
6         (CHILDRENS CUTLERY DOLLY GIRL )            0.058568   
7           (CHILDRENS CUTLERY SPACEBOY )            0.060738   
32        (SET/6 RED SPOTTY PAPER PLATES)            0.117137   
33          (SET/6 RED SPOTTY PAPER CUPS)            0.108460   
34            (ALARM CLOCK BAKELIKE RED )            0.062907   
35            (ALARM CLOCK BAKELIKE PINK)            0.067245   
36           (ALARM CLOCK BAKELIKE GREEN)            0.062907   
46        (SET/6 RED SPOTTY PAPER PLATES)            0.086768   
47          (SET/6 RED SPOTTY PAPER CUPS)            0.086768   
48  (SET/20 RED RETROSPOT PAPER NAPKINS )            0.104121   

    consequent support   support  confidence       lift  leverage  conviction  
2             0.080260  0.067245    0.815789  10.164296  0.060629    4.992873  
3             0.082430  0.067245    0.837838  10.164296  0.060629    5.658351  
6             0.060738  0.054230    0.925926  15.244709  0.050673   12.680043  
7             0.058568  0.054230    0.892857  15.244709  0.050673    8.786696  
32            0.108460  0.104121    0.888889   8.195556  0.091417    8.023861  
33            0.117137  0.104121    0.960000   8.195556  0.091417   22.071584  
34            0.080260  0.054230    0.862069  10.740913  0.049181    6.668113  
35            0.086768  0.054230    0.806452   9.294355  0.048395    4.718366  
36            0.082430  0.054230    0.862069  10.458258  0.049045    6.652386  
46            0.108460  0.084599    0.975000   8.989500  0.075188   35.661605  
47            0.117137  0.084599    0.975000   8.323611  0.074435   35.314534  
48            0.112798  0.084599    0.812500   7.203125  0.072854    4.731743

By looking at the rules, it seems that the **green and red alarm clocks are purchased together and the red paper cups, napkins and plates are purchased together** in a manner that is higher than the overall probability would suggest.
At this point, you may want to look at how much opportunity there is to use the popularity of one product to drive sales of another. For instance, we can see that we sell 340 Green Alarm clocks but only 316 Red Alarm Clocks so maybe we can drive more Red Alarm Clock sales through recommendations?

In [14]:
#lets look at a store in german
german = pd.read_csv('german_data.csv')
print(german.shape)
german.head()

(9495, 8)


InvoiceNo StockCode                          Description  Quantity  \
0    536527     22809              SET OF 6 T-LIGHTS SANTA         6   
1    536527     84347  ROTATING SILVER ANGELS T-LIGHT HLDR         6   
2    536527     84945   MULTI COLOUR SILVER T-LIGHT HOLDER        12   
3    536527     22242        5 HOOK HANGER MAGIC TOADSTOOL        12   
4    536527     22244           3 HOOK HANGER MAGIC GARDEN        12   

        InvoiceDate  UnitPrice  CustomerID  Country  
0  01-12-2010 13:04       2.95       12662  Germany  
1  01-12-2010 13:04       2.55       12662  Germany  
2  01-12-2010 13:04       0.85       12662  Germany  
3  01-12-2010 13:04       1.65       12662  Germany  
4  01-12-2010 13:04       1.95       12662  Germany

In [15]:
basket2 = (german
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

basket_sets2 = basket2.applymap(encode_units)
basket_sets2.drop('POSTAGE', inplace=True, axis=1)
frequent_itemsets2 = apriori(basket_sets2, min_support=0.05, use_colnames=True)
rules2 = association_rules(frequent_itemsets2, metric="lift", min_threshold=1)

rules_gen = rules2[ (rules2['lift'] >= 1) &
        (rules2['confidence'] >= 0.5)]
rules_gen.sort_values(by='lift', ascending=False)

antecedants  \
1       (PLASTERS IN TIN CIRCUS PARADE )   
4  (ROUND SNACK BOXES SET OF4 WOODLAND )   
5   (ROUND SNACK BOXES SET OF 4 FRUITS )   
7                  (SPACEBOY LUNCH BOX )   
3     (PLASTERS IN TIN WOODLAND ANIMALS)   

                             consequents  antecedent support  \
1     (PLASTERS IN TIN WOODLAND ANIMALS)            0.087894   
4   (ROUND SNACK BOXES SET OF 4 FRUITS )            0.185738   
5  (ROUND SNACK BOXES SET OF4 WOODLAND )            0.119403   
7  (ROUND SNACK BOXES SET OF4 WOODLAND )            0.077944   
3  (ROUND SNACK BOXES SET OF4 WOODLAND )            0.104478   

   consequent support   support  confidence      lift  leverage  conviction  
1            0.104478  0.051410    0.584906  5.598383  0.042227    2.157395  
4            0.119403  0.099502    0.535714  4.486607  0.077325    1.896670  
5            0.185738  0.099502    0.833333  4.486607  0.077325    4.885572  
7            0.185738  0.053068    0.680851  3.665653  0.038591    2.551354  
3            0.185738  0.056385    0.539683  2.905612  0.036979    1.768914

In [ ]:
#you can do analysis on data in different contries and get rules that are interesting